In [1]:
import os
import json  # Added for saving results
import time  # Added for timing
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- Setup and API Key Loading ---
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LLM_MODEL = "gpt-4o"

if not OPENAI_API_KEY:
    print("Error: OPENAI_API_KEY not found. Please set it in your .env file.")
else:
    print(f"OpenAI API Key loaded. Using model: {LLM_MODEL}")

c:\Users\ngzhi\Documents\Year 4 Sem 1\DSA4213\DSA4213-SEC-Fillings-Chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OpenAI API Key loaded. Using model: gpt-4o


In [2]:
# --- Define the "Benchmark" Chain (No RAG) ---

# 1. Initialize the LLM (same as your RAG pipeline)
llm = ChatOpenAI(
    model=LLM_MODEL,
    api_key=OPENAI_API_KEY
)

# 2. Define the Prompt Template
# This prompt asks the LLM to answer from its own knowledge.
prompt_template_str = """You are a helpful financial analyst assistant.

Answer the following question regarding 10q reports in SEC Fillings to the best of your ability based on your general, pre-trained knowledge.

Question: {question}

Answer:"""

prompt = ChatPromptTemplate.from_template(prompt_template_str)

# 3. Define a simple string output parser
output_parser = StrOutputParser()

# 4. Create the "benchmark" chain
benchmark_chain = prompt | llm | output_parser
print("Benchmark (non-RAG) chain created successfully.")

Benchmark (non-RAG) chain created successfully.


In [3]:
# --- Run Queries and Save Answers ---
if __name__ == "__main__":

    # Define the output folder
    DATA_FOLDER = "data"
    if not os.path.exists(DATA_FOLDER):
        os.makedirs(DATA_FOLDER)
        print(f"Created data folder: {DATA_FOLDER}")

    # The 6 standard questions from your base_RAG notebook
    queries_to_run = [
        {"question": "What are the main risk factors mentioned by each companies?", "ticker_filter": None},
        {"question": "What risks did Apple disclose in their latest 10-Q?", "ticker_filter": None},
        {"question": "Compare the revenue trends of NVIDIA and AMD", "ticker_filter": None},
        {"question": "What was Tesla's R&D spending in the latest quarter?", "ticker_filter": None},
        {"question": "How has Microsoft's operating income changed over the last year?", "ticker_filter": None},
        {"question": "What was the gross profit margin for all companies?", "ticker_filter": None},
    ]

    print(f"\nRunning {len(queries_to_run)} queries with (Model 0) Baseline No-RAG...")
    
    # This list will hold all the results
    all_results = []

    for query in queries_to_run:
        print(f"\n--- Running Query: {query['question']} ---")
        
        start_time = time.time()
        
        # Run the query and get the result
        answer = benchmark_chain.invoke({"question": query["question"]})
        
        time_taken = time.time() - start_time
        
        print(f"ANSWER (from gpt-4o's internal knowledge):\n{answer}")
        print(f"Time taken: {time_taken:.2f}s")
        
        # Create a dictionary for this query and its result
        query_output = {
            "question": query["question"],
            "ticker_filter_manual": query["ticker_filter"], # Keep format consistent
            "response": {
                "answer": answer,
                "sources": [], # No sources for a non-RAG model
                "time_taken": time_taken
            }
        }
        all_results.append(query_output)

    # After all queries are done, save the entire list to one file
    save_path = os.path.join(DATA_FOLDER, "baseline_no_RAG_results.json")
    print(f"\n--- All queries complete. Saving all results to: {save_path} ---")

    try:
        with open(save_path, 'w', encoding='utf-8') as f:
            json.dump(all_results, f, indent=4)
        print("✓ Successfully saved all answers.")
    except Exception as e:
        print(f"✗ Failed to save results: {e}")

    print("\nBaseline run finished.")


Running 6 queries with (Model 0) Baseline No-RAG...

--- Running Query: What are the main risk factors mentioned by each companies? ---
ANSWER (from gpt-4o's internal knowledge):
As a financial analyst assistant, it's important to note that each company's risk factors, disclosed in their 10-Q reports, vary based on their specific industry, market conditions, and corporate strategy. However, there are common categories of risk factors that many companies typically report in their SEC filings. Here are some of these categories:

1. **Market Risk**: This includes risks related to economic conditions, changes in consumer preferences, and competition. Companies often discuss how fluctuations in supply and demand, as well as competitive pressures, may impact their business.

2. **Operational Risks**: These are risks associated with the company's internal processes, systems, and personnel. It includes supply chain disruptions, dependency on key suppliers or customers, and challenges related 